## Gym Spaces

Weitere Begriffe: Spaces
- Discrete: X verschiedene Werte
- Box: Bereich von Werten (X-Y)
- Sequence: Beliebig viele Werte (effektiv eine Liste)

In [68]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence
import numpy as np
from typing import Optional

In [11]:
d = Discrete(3)
for x in range(10):
    print(d.sample())

2
0
2
1
1
1
0
1
2
2


In [13]:
b = Box(0, 1)
b.sample()

array([0.27835065], dtype=float32)

In [17]:
Box(0, 1, shape=(3,)).sample()

array([0.45214197, 0.5838016 , 0.9584438 ], dtype=float32)

In [18]:
Box(0, 1, shape=(3, 3)).sample()

array([[0.51214504, 0.07954149, 0.2866445 ],
       [0.66909397, 0.6246519 , 0.43020725],
       [0.9470782 , 0.4826286 , 0.65197414]], dtype=float32)

In [19]:
Box(0, 100, shape=(3, 3), dtype=int).sample()

array([[19, 31, 75],
       [26, 90, 86],
       [82, 59,  7]])

In [40]:
Sequence(Box(0, 1)).sample()

(array([0.5819444], dtype=float32),
 array([0.38592443], dtype=float32),
 array([0.35165536], dtype=float32),
 array([0.97796726], dtype=float32),
 array([0.38957372], dtype=float32),
 array([0.15212134], dtype=float32),
 array([0.70136464], dtype=float32))

## Eigene Umgebung

Eine eigene Umgebung muss drei Dinge enthalten:
- Action Space
    - Möglichen Aktionen des Agenten

- Observation Space
    - Daten die für die Beobachtung verwendet werden

- State
    - Zustand der Umgebung
 
Unsere Umgebung muss von env erben

---

Beispiel: Autofahrer

Geschwindigkeit soll zwischen 45 und 55 km/h gehalten werden

In [93]:
class CarDriver(gym.Env):
    def __init__(self):
        self.action_space = Discrete(3)  # Drei Aktionen: Bremsen, Beschleunigen, Nichts
        self.observation_space = Box(0, 100, dtype=np.int8)  # Geschwindigkeitsgrenzen
        self.state = 40 + np.random.randint(10)
        self.drive_length = 60

    def step(self, action):
        # Action verwenden
        # 0: +-0km/h
        # 1: -3km/h
        # 2: +3km/h
        if action == 1:
            self.state -= 3
        if action == 2:
            self.state += 3

        if self.state >= 45 and self.state <= 55:
            reward = 1
        else:
            reward = -1

        self.drive_length -= 1
        
        # Hier kommen 5 Rückgabewerte heraus: Den Game State (Observation Space), die Belohnung, Verloren Ja/Nein, Spiel zu Ende (optional), Info: Beliebige Daten
        return np.array(self.state, dtype=np.int8).reshape(1,), reward, self.drive_length <= 0, False, {}

    def reset(self, *, seed: Optional[int] = None, options: Optional[dict] = None):
        self.state = 40 + np.random.randint(10)
        self.drive_length = 60
        return (np.array(self.state, dtype=np.int8).reshape(1, ), {})

In [53]:
def testEnv(env, durchgaenge):
    for d in range(durchgaenge):
        state = env.reset()
        score = 0
        terminate = False

        # Diese Schleife spielt das Spiel
        while not terminate:
            action = env.action_space.sample()
            state, reward, terminate, done, info = env.step(action)
            score += reward
        print(f"Durchgang {d + 1}: Score {score}")

In [94]:
env = CarDriver()

In [56]:
testEnv(env, 10)

Durchgang 1: Score -38
Durchgang 2: Score -34
Durchgang 3: Score -32
Durchgang 4: Score 40
Durchgang 5: Score -50
Durchgang 6: Score 0
Durchgang 7: Score 14
Durchgang 8: Score -16
Durchgang 9: Score -12
Durchgang 10: Score 24


## Env Checker

In [63]:
from stable_baselines3.common.env_checker import check_env

In [95]:
check_env(env, warn=True)

## Modell erstellen

In [96]:
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [97]:
env = CarDriver()
env = DummyVecEnv([lambda: env])
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device


In [98]:
model.learn(total_timesteps=200000)

-----------------------------
| time/              |      |
|    fps             | 1996 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1420        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010081819 |
|    clip_fraction        | 0.134       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.00208     |
|    learning_rate        | 0.0003      |
|    loss                 | 27.6        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0137     |
|    value_loss           | 67.4        |
-----------------------------------------
----------------------------------

In [99]:
model.save("Models/CarDriver")

In [105]:
env = CarDriver()

In [106]:
model = PPO.load("Models/CarDriver.zip", env=env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [109]:
def testEnvPredict(env, durchgaenge):
    for d in range(durchgaenge):
        state = env.reset() # Am Anfang des Durchgangs die Umgebung zurücksetzen
        score = 0
        terminate = False
        done = False

        while not (terminate or done):
            if len(state) == 2:
                action = model.predict(np.array(state[0]))
            else:
                action = model.predict(np.array(state))
            state, reward, terminate, done, info = env.step(action[0])  # Führe einen Step mit der Random Action aus
            # Hier kommen 5 Rückgabewerte heraus: Den Game State (Observation Space), die Belohnung, Verloren Ja/Nein, Spiel zu Ende (optional), Info: Beliebige Daten
            score += reward
        print(f"Durchgang {d + 1}: Score {score}")
    env.close()

In [111]:
testEnvPredict(env, 10)

Durchgang 1: Score 58
Durchgang 2: Score 44
Durchgang 3: Score 60
Durchgang 4: Score 58
Durchgang 5: Score 58
Durchgang 6: Score 56
Durchgang 7: Score 36
Durchgang 8: Score 60
Durchgang 9: Score 60
Durchgang 10: Score 60
